<a href="https://colab.research.google.com/github/ulfboge/temporal-landcover-vectorizer/blob/main/scripts/python/stratified_biomass_sampler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spatial Data Sampling and Integration

This notebook:
1. Processes multiple CSV files containing temporal spatial data
2. Creates two unified long-format tables:
   - One without coordinates (original format)
   - One with coordinates (x_coord, y_coord)
3. Samples biomass data in equal intervals
4. Integrates corresponding NDVI and NDFI values

## Setup
First, let's import required libraries and set up directories.

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Set the working directory
csv_dir = "csv"  # Modify this path as needed
output_dir = "output"  # Modify this path as needed
os.makedirs(output_dir, exist_ok=True)

## Define Processing Functions
Create function to process CSV files into long format, with option to include coordinates.

In [ ]:
def process_csv_to_long_format(file_path, include_coords=False):
    # Extract the data type from filename (NDVI, NDFI, or biomass)
    data_type = Path(file_path).stem.split('_')[0].upper()

    # Read CSV
    df = pd.read_csv(file_path)

    # Identify year columns (starting with 'y')
    year_cols = [col for col in df.columns if col.startswith('y')]

    # Define id_vars based on whether coordinates should be included
    id_vars = ['pixel_id']
    if include_coords:
        id_vars.extend(['x_coord', 'y_coord'])

    # Melt the dataframe to long format
    long_df = pd.melt(
        df,
        id_vars=id_vars,
        value_vars=year_cols,
        var_name='year',
        value_name=data_type
    )

    # Convert year format from 'y2013' to '2013'
    long_df['year'] = long_df['year'].str.replace('y', '')

    return long_df

## Process CSV Files
Process all CSV files and create two versions of each dataset - with and without coordinates.

In [ ]:
# Process all CSV files - create two versions
dfs = {'with_coords': {}, 'without_coords': {}}
for file in os.listdir(csv_dir):
    if file.endswith('.csv'):
        file_path = os.path.join(csv_dir, file)
        data_type = Path(file).stem.split('_')[0].upper()
        dfs['with_coords'][data_type] = process_csv_to_long_format(file_path, include_coords=True)
        dfs['without_coords'][data_type] = process_csv_to_long_format(file_path, include_coords=False)
        print(f"Processed {file}")

## Sample Biomass Data
Sample 100 points from biomass data with equal distribution across intervals.

In [ ]:
# Define intervals and sample biomass data
intervals = [(5, 34), (35, 64), (65, 94), (95, 124), (125, 150)]
samples_per_interval = 20  # 20 samples per interval = 100 total

# Filter and sample biomass data
biomass_df = dfs['without_coords']['BIOMASS']  # Use either version for sampling
sampled_pixels = []

for start, end in intervals:
    # Filter data within interval
    interval_data = biomass_df[
        (biomass_df['BIOMASS'] >= start) &
        (biomass_df['BIOMASS'] <= end)
    ]

    # Sample from this interval
    if len(interval_data) >= samples_per_interval:
        sampled = interval_data.sample(n=samples_per_interval, random_state=42)
    else:
        sampled = interval_data  # Take all available if less than needed
        print(f"Warning: Only {len(interval_data)} samples available for interval {start}-{end}")

    sampled_pixels.extend(sampled['pixel_id'].unique())

print(f"Total unique pixels sampled: {len(set(sampled_pixels))}")

## Filter and Merge Data
Create filtered datasets and merge them to create final results.

In [ ]:
# Create two versions of the final result
filtered_dfs = {
    'with_coords': {},
    'without_coords': {}
}

# Filter each dataset for sampled pixels
for version in ['with_coords', 'without_coords']:
    for data_type, df in dfs[version].items():
        filtered_dfs[version][data_type] = df[df['pixel_id'].isin(sampled_pixels)]

# Create and save both versions
results = {}

# Version without coordinates (original)
result_without_coords = filtered_dfs['without_coords']['BIOMASS'][['pixel_id', 'year', 'BIOMASS']]
for data_type in ['NDVI', 'NDFI']:
    if data_type in filtered_dfs['without_coords']:
        result_without_coords = result_without_coords.merge(
            filtered_dfs['without_coords'][data_type][['pixel_id', 'year', data_type]],
            on=['pixel_id', 'year'],
            how='left'
        )
results['without_coords'] = result_without_coords.sort_values(['pixel_id', 'year'])

# Version with coordinates
result_with_coords = filtered_dfs['with_coords']['BIOMASS'][['pixel_id', 'x_coord', 'y_coord', 'year', 'BIOMASS']]
for data_type in ['NDVI', 'NDFI']:
    if data_type in filtered_dfs['with_coords']:
        result_with_coords = result_with_coords.merge(
            filtered_dfs['with_coords'][data_type][['pixel_id', 'year', data_type]],
            on=['pixel_id', 'year'],
            how='left'
        )
results['with_coords'] = result_with_coords.sort_values(['pixel_id', 'year'])

## Save Results and Display Summary
Save both versions of the results and display summary statistics.

In [ ]:
# Save both versions
output_path_without_coords = os.path.join(output_dir, 'sampled_data.csv')
output_path_with_coords = os.path.join(output_dir, 'sampled_data_with_coords.csv')

results['without_coords'].to_csv(output_path_without_coords, index=False)
results['with_coords'].to_csv(output_path_with_coords, index=False)

print(f"\nResults saved to:")
print(f"1. Without coordinates: {output_path_without_coords}")
print(f"2. With coordinates: {output_path_with_coords}")

# Print summary statistics
print("\nSummary of sampled data:")
print(f"Total unique pixels: {results['with_coords']['pixel_id'].nunique()}")
print(f"Years covered: {sorted(results['with_coords']['year'].unique())}")
print("\nValue ranges:")
for column in ['BIOMASS', 'NDVI', 'NDFI']:
    if column in results['with_coords'].columns:
        print(f"{column}:")
        print(f"  Min: {results['with_coords'][column].min():.2f}")
        print(f"  Max: {results['with_coords'][column].max():.2f}")
        print(f"  Mean: {results['with_coords'][column].mean():.2f}")